In [1]:
import spacy
import random
import pandas as pd
import json
from urllib.request import urljoin 
from bs4 import BeautifulSoup 
import requests 
from urllib.request import urlparse 
from urllib.error import HTTPError
from urllib.error import URLError
import openpyxl
from openpyxl import Workbook
from openpyxl.styles import Color, PatternFill, Font, Border
from openpyxl.styles import colors
from openpyxl.cell import Cell
from openpyxl.styles import Color, Fill
from openpyxl.cell import Cell

In [2]:
from google.colab import drive
drive.mount("my-drive")

Mounted at my-drive


In [3]:
nlp_model=spacy.load('my-drive/My Drive/nlp_model')

In [4]:
def selected_tags(tag):
  return tag.name!='a' and tag.name!='style' and tag.name!='script' and tag.name!='title'
def extract_info(url):
  try:
    page=requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    content=soup.find_all(selected_tags)
    all_links=soup.find_all('a')
    contentstr=', '.join([str(alltext.text).strip() for alltext in content if '\n' not in alltext.text])
    email=set()
    personal_website=set()
    print('Scraping Url: '+url)
    for link in all_links:
      if '@' in link.text and '.' in link.text:
        email.add(link.text.strip())
      elif 'email' in str(link.text).lower():
        email.add(link.get('href').strip())
        print(link.text+'\t'+link.get('href'))
      if 'website' in str(link.text).lower() and (('https' in link.get('href')) or ('http' in link.get('href'))):
        personal_website.add(link.get('href').strip())
        print(link.text+'\t'+link.get('href'))
      elif 'profile' in str(link.text).lower() and (('https' in link.get('href')) or ('http' in link.get('href'))):
        personal_website.add(link.get('href').strip())
        print(link.text+'\t'+link.get('href'))
    return contentstr,list(email),list(personal_website)
  except HTTPError as e:
    print("404 Not Found")
    return "",[],[]
  except URLError as e:
    print("The server can't be reached")
    return "",[],[]
  except:
    print("Some unexpected error occurred")
    return "",[],[]

In [10]:
def store_data(path,df,df1):

  if '.xlsx' in path:
    path=path.replace('.xlsx','_updated.xlsx')
  elif '.csv' in path:
    path=path.replace('.xlsx','_updated.xlsx')

  wb = openpyxl.Workbook() 
  sheet = wb.active 
  c_idx = 1
  for col in df.columns:
    c1 = sheet.cell(1,c_idx)
    c1.value = str(col)
    c_idx = c_idx+1

  c1 = sheet.cell(1,c_idx)
  c1.value = "Change"

  for r in range(len(df.index)):
    for c in range(len(df.columns)):
      cmp1 = (df.iloc[r][c])
      cmp2 = (df1.iloc[r][c])
      cell1  = sheet.cell(r+2,c+1)
      if df1.iloc[r][len(df.columns)]=="No" or str(cmp1).strip()==str(cmp2).strip():
        cell1.value = "Not Found" if str(cmp1).strip()=="" else cmp1
      else:
        cell1.value = "Not Found" if str(cmp2).strip()=="" else cmp2
        string = str(cell1)
        string = string[14:]
        string = string[:-1]
        # print(string)

        redFill = PatternFill(start_color='FFFF0000',
                    end_color='FFFF0000',
                    fill_type='solid')

        sheet[string].fill = redFill    
    cell2 = sheet.cell(r+2,len(df.columns)+1)
    cell2.value = df1.iloc[r][len(df.columns)]  

  wb.save(path)

In [13]:
def update_info(path,df,content_df):
  if '.xlsx' in path:
    copy_df=pd.read_excel(path)
  elif '.csv' in path:
    copy_df=pd.read_csv(path)
  df['Change']='No'
  ent_weights={'Specialization':10,'Address':30,'Experience':20,'Email':50,'MobileNumber':40,'Qualification':20,'Designation':20,'PersonalWebsite':25,'Department':20,'Degree':20}
  for i in range(len(df)):
    total_weight=0
    Specialization=''
    Address=''
    Experience=''
    MobileNumber=''
    Qualification=''
    Designation=''
    Department=''
    Degree=''
    print('Predicting')
    doc=nlp_model(str(content_df.at[i,'content']))
    for ent in doc.ents:
      if len(ent.text)<3:
        continue
      print(ent.text+'\t'+ent.label_)
      if ent.label_=='Specialization':
        if ent.text.lower() not in str(df.at[i,'Area of Specialization(in words)']).lower() and ent.text.lower() not in Specialization.lower():
          Specialization=Specialization+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Address':
        if ent.text.lower() not in str(df.at[i,'Corresponding Address']).lower() and ent.text.lower() not in Address.lower():
          Address=Address+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Experience':
        if ent.text.lower() not in str(df.at[i,'Experience']).lower() and ent.text.lower() not in Experience.lower():
          Experience=Experience+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='MobileNumber':
        if ent.text.lower() not in str(df.at[i,'Mobile No']).lower() and ent.text.lower() not in MobileNumber.lower():
          MobileNumber=MobileNumber+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Qualification':
        if ent.text.lower() not in str(df.at[i,'Qualification']).lower() and ent.text.lower() not in Qualification.lower():
          Qualification=Qualification+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Designation':
        if ent.text.lower() not in str(df.at[i,'Designation']).lower() and ent.text.lower() not in Designation.lower():
          Designation=Designation+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Department':
        if ent.text.lower() not in str(df.at[i,'Department']).lower() and ent.text.lower() not in Department.lower():
          Department=Department+', '+ent.text
          total_weight+=ent_weights[ent.label_]
      elif ent.label_=='Degree':
        if ent.text.lower() not in str(df.at[i,'Doctorate Degree']).lower() and ent.text.lower() not in Degree.lower():
          Degree=Degree+', '+ent.text
          total_weight+=ent_weights[ent.label_]
    if content_df.at[i,'email'] not in df.at[i,'E-mail']:
      total_weight+=ent_weights['Email']
    if content_df.at[i,'personal_website'] not in df.at[i,'Personal Website']:
      total_weight+=ent_weights['PersonalWebsite'] 
    print(total_weight)
    if total_weight>150:
      print('Changing')
      df.at[i,'Change']='Yes'
      if len(Specialization)!=0:
        if 'not found' in df.at[i,'Area of Specialization(in words)'].lower():
          df.at[i,'Area of Specialization(in words)']=Specialization
        else:
          df.at[i,'Area of Specialization(in words)']=df.at[i,'Area of Specialization(in words)']+Specialization
      if len(Address)!=0:
        if 'not found' in df.at[i,'Corresponding Address'].lower():
          df.at[i,'Corresponding Address']=Address
        else:
          df.at[i,'Corresponding Address']=df.at[i,'Corresponding Address']+Address
      if len(Experience)!=0:
        if 'not found' in df.at[i,'Experience'].lower():
          df.at[i,'Experience']=Experience
        else:
          df.at[i,'Experience']=df.at[i,'Experience']+Experience
      if len(MobileNumber)!=0:
        if 'not found' in df.at[i,'Mobile No'].lower():
          df.at[i,'Mobile No']=MobileNumber
        else:
          df.at[i,'Mobile No']=df.at[i,'Mobile No']+MobileNumber
      if len(Qualification)!=0:
        if 'not found' in df.at[i,'Qualification'].lower():
          df.at[i,'Qualification']=Qualification
        else:
          df.at[i,'Qualification']=df.at[i,'Qualification']+Qualification
      if len(Designation)!=0:
        if 'not found' in df.at[i,'Designation'].lower():
          df.at[i,'Designation']=Designation
        else:
          df.at[i,'Designation']=df.at[i,'Designation']+Designation
      if len(Department)!=0:
        if 'not found' in df.at[i,'Department'].lower():
          df.at[i,'Department']=Department
        else:
          df.at[i,'Department']=df.at[i,'Department']+Department
      if len(Degree)!=0:
        if 'not found' in df.at[i,'Doctorate Degree'].lower():
          df.at[i,'Doctorate Degree']=Degree
        else:
          df.at[i,'Doctorate Degree']=df.at[i,'Doctorate Degree']+Degree
      if content_df.at[i,'email'] not in df.at[i,'E-mail']:
        df.at[i,'E-mail']=content_df.at[i,'email']
      if content_df.at[i,'personal_website'] not in df.at[i,'Personal Website']:
        if 'not found' in df.at[i,'Personal Website'].lower():
          df.at[i,'Personal Website']=content_df.at[i,'personal_website']
        else:
          df.at[i,'Personal Website']=df.at[i,'Personal Website']+', '+content_df.at[i,'personal_website']
  store_data(path,copy_df,df)

In [7]:
def update_changes(path,df):
  urls=list(df['URL'])
  info_dict={'content':[],'email':[],'personal_website':[]}
  for i in range(len(urls)):
    url_list=set(urls[i].split(','))
    url_list=list(url_list)
    content=''
    email=[]
    personal_website=[]
    for url in url_list:
      if '.edu' not in url:
        continue
      url=url.strip()
      content1,email1,personal_website1=extract_info(url)      
      content=content+', '+content1
      email.extend(email1)
      personal_website.extend(personal_website1)
    email=set(email)
    email=list(email)
    personal_website=set(personal_website)
    personal_website=list(personal_website)
    email_str=', '.join([str(everymail) for everymail in email])
    personal_website_str=', '.join([str(everywebsite) for everywebsite in personal_website])
    print('Email: '+email_str)
    print('Website: '+personal_website_str)
    if len(content)>1000000:
      content=content[:900000]
    info_dict['content'].append(content)
    info_dict['email'].append(email_str)
    info_dict['personal_website'].append(personal_website_str)
  content_df=pd.DataFrame.from_dict(info_dict)
  print(len(df))
  print(len(content_df))
  update_info(path,df,content_df)

In [8]:
def read_file(path,option):
  df=''
  if path.endswith('.csv'):
    df=pd.read_csv(path)
  elif path.endswith('.xlsx'):
    df=pd.read_excel(path)
  else:
    print('This file format is not accepted')
    return
  if option==1:
    update_changes(path,df)
  elif option==2:
    update_changes(path,df)
    # find_new_faculty(path,df)
  elif option==3:
    update_changes(path,df)
    # find_new_faculty(path,df)

In [14]:
print("Enter name of file:")
filename=input()
basepath='my-drive/My Drive/'
read_file(basepath+filename,1)
# 1 california institute of technology.xlsx

Enter name of file:
1 california institute of technology.xlsx
Scraping Url: http://www.pellegrino.caltech.edu/manan-arya-bio/
Email: 
Website: 
Scraping Url: https://directory.caltech.edu/personnel/vratnasw
Scraping Url: http://www.njit.edu/features/alumni/ratnaswamy.php
Email: 
Website: 
Scraping Url: http://www.kochmann.caltech.edu/group_students.html
Scraping Url: https://directory.caltech.edu/personnel/vvidyasa


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping Url: https://www.math.cmu.edu/CNA/Summer16/upload/slides/Vidyasagar.pdf
Email: 
Website: 
Scraping Url: https://directory.caltech.edu/personnel/skumar2
Email: 
Website: 
Scraping Url: http://www.kochmann.caltech.edu/group_students.html
Scraping Url: http://caltech.academia.edu/NeelNadkarni
Scraping Url: https://directory.caltech.edu/personnel/nnadkarn
Email: 
Website: 
Scraping Url: http://www.systems.caltech.edu/dsp/
Scraping Url: http://systems.caltech.edu/dsp/students/srikanth/
Scraping Url: https://directory.caltech.edu/personnel/stenneti
Email: 
Website: 
Scraping Url: https://directory.caltech.edu/personnel/sjain2
Personal Website	http://paradise.caltech.edu/~sidjain
Scraping Url: http://caltech.academia.edu/SiddharthJain
Email: 
Website: http://paradise.caltech.edu/~sidjain
Scraping Url: https://directory.caltech.edu/personnel/itembhek
Scraping Url: http://www.kochmann.caltech.edu/group_students.html
Email: 
Website: 
Some unexpected error occurred
Some unexpected error

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping Url: http://labs.vtc.vt.edu/hnl/articles/web_cv/Meghana_Bhatt_CV.pdf
Email: 
Website: 
Scraping Url: http://www.its.caltech.edu/~kthyagar/
Some unexpected error occurred
Email: 
Website: 
Scraping Url: https://www.hss.caltech.edu/content/anita-tusche
Email: 
Website: 
Scraping Url: https://directory.caltech.edu/static/personnel/caltech_personnel_directory.pdf
Email: 
Website: 
Scraping Url: http://thesis.library.caltech.edu/1816/
Scraping Url: https://copilot.caltech.edu/documents/125-oe_pc_nl_absorption_dispersion.pdf
Email: opainter@caltech.edu
Website: 
Scraping Url: http://thesis.library.caltech.edu/1663/
Email: 
Website: 
Scraping Url: http://groups.csail.mit.edu/synbio/people/deepak-mishra/
Email: 
Website: 
Scraping Url: http://www.astro.caltech.edu/~spt/
Email: 
Website: 
Scraping Url: https://directory.caltech.edu/personnel/docpriya
Scraping Url: http://ucsd.academia.edu/PriyaChoudhry
Email: 
Website: 
Scraping Url: https://labcit.ligo.caltech.edu/~rana/code/index.htm

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Scraping Url: http://www.submm.caltech.edu/~goutam/academic_links/cv.pdf
Email: 
Website: 
Scraping Url: http://www.astro.caltech.edu/~mansi/about_me.html
Scraping Url: http://www.caltech.edu/news/hunting-ephemeral-cosmic-flashes-conversation-mansi-kasliwal-49782
Scraping Url: https://pma.caltech.edu/content/mansi-m-kasliwal
Email: debwms@caltech.edu
Website: 
Scraping Url: https://pma.caltech.edu/content/dinakar-ramakrishnan
Scraping Url: https://directory.caltech.edu/personnel/dinakar
Research Website	http://www.math.caltech.edu/~dinakar/
Email: dinakar@caltech.edu
Website: http://www.math.caltech.edu/~dinakar/
Scraping Url: http://vivo.cornell.edu/display/KulkarniShri
Scraping Url: http://www.gps.caltech.edu/content/shrinivas-r-shri-kulkarni
Website Feedback	https://form.jotform.com/82387643111152
Scraping Url: https://pma.caltech.edu/content/shrinivas-r-shri-kulkarni
Scraping Url: http://www.gps.caltech.edu/content/srk/cv
Website Feedback	https://form.jotform.com/82387643111152
Ema